In [72]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import networkx as nx

In [73]:
path1 = input("Please enter the path of canceled.csv file: ")
path2 = input("Please enter the path of flights.csv file: ")
path3 = input("Please enter the path of passengers.csv file: ")
path4= input("Please enter the desired path of output file stats.csv file: ")
path5 = input("Please enter the desired path of output file allot.csv file: ")

canceled = pd.read_csv(path1)
flights = pd.read_csv(path2)
passengers = pd.read_csv(path3)

In [74]:
import time
start_time = time.time()

In [75]:
flights.insert(4,'OCCUPIED','')
flights.insert(5,'VACANT','')

In [76]:
for i in flights.loc[:,'FID']:
      flights.at[i,'OCCUPIED']=passengers['FID'].value_counts()[i]
      flights.at[i,'VACANT']=flights.at[i,'CAPACITY']-flights.at[i,'OCCUPIED']


In [77]:
s=canceled.iloc[:,0]
sum=0;

for i in s:
  # print(i)
  sum+= passengers['FID'].value_counts()[i]
dict = {
    "Affected":[sum]
}
df=pd.DataFrame(dict)
df.to_csv('stats.csv')

In [78]:
G = nx.from_pandas_edgelist(
    flights,
    source="DEP",
    target="ARR",
    edge_key="FID",
    edge_attr=["DEP_TIME", "ARR_TIME","FID","CAPACITY"],
    create_using=nx.MultiDiGraph(),
)

In [79]:
for fid in canceled.iloc[:,0]:
  G.remove_edge(flights.loc[fid,"DEP"], flights.loc[fid,"ARR"], key=fid)

In [80]:
def get_capacity(path):
    min_capacity = float('inf') 
    for edge in path:
      _, _, fid = edge
      cap = flights.loc[fid, 'VACANT']
      min_capacity = min(min_capacity, cap)
    return min_capacity

In [81]:
rows = len(canceled)
all_valid_paths = []

for i in range(rows):
    fid = canceled.iloc[i, 0]
    dep = flights.loc[fid, "DEP"]
    arr = flights.loc[fid, "ARR"]
    dept = flights.loc[fid, "DEP_TIME"]
    
    valid_paths = []
    sum_paths = 0

    for path in sorted(nx.all_simple_edge_paths(G, dep, arr, 3)):
        if len(path) == 3 and get_capacity(path) >0 :
            if (flights.loc[path[1][2], "DEP_TIME"] - flights.loc[path[0][2], "ARR_TIME"] >= 7200) and \
               (flights.loc[path[2][2], "DEP_TIME"] - flights.loc[path[1][2], "ARR_TIME"] >= 7200):
                valid_paths.append(path)
                sum_paths += 1
        elif len(path) == 2 and get_capacity(path) >0:
            if (flights.loc[path[1][2], "DEP_TIME"] - flights.loc[path[0][2], "ARR_TIME"] >= 7200):
                valid_paths.append(path)
                sum_paths += 1
        elif len(path) == 1 and get_capacity(path) >0 :
            valid_paths.append(path)
            sum_paths += 1

    def custom_sort(path):
        arr_time_diff = flights.loc[path[-1][2], "ARR_TIME"] - flights.loc[fid, "ARR_TIME"]
        sign_priority = 1 if arr_time_diff >= 0 else -1
        length_priority = len(path)
        magnitude_priority = abs(arr_time_diff)
        return ( sign_priority,length_priority,magnitude_priority)

    valid_paths = sorted(valid_paths, key=custom_sort)
    all_valid_paths.append(valid_paths)


In [82]:
order = canceled.iloc[:,0].tolist()
filtered_passengers = passengers[passengers['FID'].isin(order)]
sorted_passengers = filtered_passengers.set_index('FID').loc[order].reset_index()
sorted_passengers_df = pd.DataFrame(sorted_passengers)
sorted_passengers_df = sorted_passengers_df.assign(edges=None)
sorted_passengers_df = sorted_passengers_df.assign(FID1=None)
sorted_passengers_df = sorted_passengers_df.assign(FID2=None)
sorted_passengers_df = sorted_passengers_df.assign(FID3=None)


In [83]:
def is_fid_in_path(fid_set, path):
    
    for edge in path:
        
            _, _, fid = edge  
            if fid in fid_set:
                return True  
    return False  

rows = len(canceled)
occupied = []

for i in range(rows):
    fid = canceled.iloc[i, 0]
    occupied.append(flights.loc[fid, "OCCUPIED"])


used_fids = set()
Allocated =0
t=0
count=0
c=0
Layover = 0
occ=occupied.copy()
total_passengers = np.sum(occupied)

for j in range(0,rows):
    if j > 0:
        c += occ[j - 1]
    else:
        c = 0
    count = c
   
   
    i = 0
    while i < len(all_valid_paths[j]) and occupied[j] > 0:
        if not is_fid_in_path(used_fids, all_valid_paths[j][i]):
            for edge in all_valid_paths[j][i]:
                _, _, fid = edge
                used_fids.add(fid)

            path_capacity = get_capacity(all_valid_paths[j][i])

            if occupied[j] >= path_capacity:
                occupied[j] -= path_capacity
                Allocated += path_capacity
                Layover+= (len(all_valid_paths[j][i])-1)*path_capacity
                t += abs(flights.loc[fid, "ARR_TIME"] - flights.loc[canceled.iloc[j, 0], "ARR_TIME"]) * path_capacity
                sorted_passengers_df.iloc[count:count + path_capacity, 2] = len(all_valid_paths[j][i])
               
                for k in range(len(all_valid_paths[j][i])):
                    sorted_passengers_df.iloc[count:count + path_capacity, 3 + k] = all_valid_paths[j][i][k][-1]
                    
                count += path_capacity
            else:
                t += abs(flights.loc[fid, "ARR_TIME"] - flights.loc[canceled.iloc[j, 0], "ARR_TIME"]) * occupied[j]
                Allocated += occupied[j]
                remaining = occupied[j]
                Layover+= (len(all_valid_paths[j][i])-1)*occupied[j]
                sorted_passengers_df.iloc[count:count + remaining, 2] = len(all_valid_paths[j][i])
                
                for k in range(len(all_valid_paths[j][i])):
                    sorted_passengers_df.iloc[count:count + remaining, 3 + k] = all_valid_paths[j][i][k][-1]
                    
                count += remaining
                occupied[j] = 0

        i += 1
sorted_passengers_df['edges']=sorted_passengers_df['edges'].fillna(0)

for idx in range(count, total_passengers):
    sorted_passengers_df.iloc[idx, 2] = 0
# sorted_passengers_df = sorted_passengers_df.drop(sorted_passengers_df.columns[0], axis=1)
sorted_passengers_df.drop(sorted_passengers_df.columns[0], axis=1, inplace=True)
sorted_passengers_df.columns = ["" for _ in sorted_passengers_df.columns]
sorted_passengers_df.to_csv('allot.csv',index=False)


C:\Users\hp\AppData\Local\Temp\ipykernel_4136\3727498791.py:69: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  sorted_passengers_df['edges']=sorted_passengers_df['edges'].fillna(0)


In [84]:
avgtime= t/Allocated
avgLay= Layover/Allocated


dict = {
    "Reallocated":[Allocated]
}
df1=pd.DataFrame(dict)
df['Reallocated']=df1
df.to_csv('stats.csv')

# df = df.drop(df.columns[0], axis=1)


dict1 = {
    "AvgLay":[avgLay]
}
df1=pd.DataFrame(dict1)
df['AvgLay']=df1
df.to_csv('stats.csv')

dict1 = {
    "AvgLay":[avgLay]
}
df1=pd.DataFrame(dict1)
df['AvgLay']=df1
df.to_csv('stats.csv')

dict1 = {
    "TimeDiff":[avgtime]
}
df1=pd.DataFrame(dict1)
df['TimeDiff']=df1
df.to_csv('stats.csv')

In [85]:
end_time = time.time()
runtm =(end_time - start_time)*1000

dict1 = {
    "SolTime":[runtm]
}
df1=pd.DataFrame(dict1)
df['SolTime']=df1
df.to_csv('stats.csv', index=False)